In [ ]:
import os
#import pandas as pd
#import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import wikipediaapi

from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from geopy import distance

from flask import Flask, request, jsonify
from flask_cors import CORS
from threading import Thread


app = Flask(__name__)
CORS(app)